In [6]:
!pip install xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/150.0 MB 560.1 kB/s eta 0:04:27
   ---------------------------------------- 0.5/150.0 MB 560.1 kB/s eta 0:04:27
   ---------------------------------------- 0.8/150.0 MB 599.2 kB/s eta 0:04:09
   ---------------------------------------- 0.8/150.0 MB 599.2 kB/s eta 0:04:09
   ---------------------------------------- 1.0/150.0 MB 662.5 kB/s eta 0:03:45
   ---------------------------------------- 1.0/150.0 MB 662.5 kB/s eta 0:03:45
   ---------------------------------------- 1.3/150.0 MB 645.3 kB/s eta 0:03:51
   ---------------------------------------- 1.6/150.0 MB 699.0 kB/s eta 0:03:33
   ---------------------------------------- 1.8/150.0 MB 745.8 kB/s eta 0:03

In [2]:
!pip install catboost


  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.5/102.4 MB 577.4 kB/s eta 0:02:57
   ---------------------------------------- 0.8/102.4 MB 744.7 kB/s eta 0:02:17
   ---------------------------------------- 0.8/102.4 MB 744.7 kB/s eta 0:02:17
   ---------------------------------------- 1.0/102.4 MB 751.1 kB/s eta 0:02:15
   ---------------------------------------- 1.0/102.4 MB 751.1 kB/s eta 0:02:15
    --------------------------------------- 1.3/102.4 MB 729.2 kB/s eta 0:02:19
    --------------------------------------- 1.6/102.4 MB 749.0 kB/s eta 0:02:15
    --------------------------------------- 1.8/102.4 MB 792.4 kB/s eta 0:02:07
    -------

In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings


In [11]:
df=pd.read_csv('Data/StudentsPerformance.csv')

In [12]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [37]:
numerical=x.select_dtypes(include=['int64', 'float64']).columns
categorical=x.select_dtypes(include=['object']).columns
print("Numerical Columns:")
print(numerical)
print("Categorical Columns:")
print(categorical)

Numerical Columns:
Index(['reading score', 'writing score'], dtype='object')
Categorical Columns:
Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course'],
      dtype='object')


In [44]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer 
numeric_trans=StandardScaler()
oh_trans=OneHotEncoder()
preprocessor=ColumnTransformer(
[
    ("OneHotEncoder",oh_trans,categorical)
    ,("StandardScaler",numeric_trans,numerical)
]
)

In [25]:
x=df.drop(columns='math score')
y=df['math score']
df

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [45]:
x=preprocessor.fit_transform(x)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [48]:
def evaluate(true,pred):
    mse=mean_squared_error(true,pred)
    r2=r2_score(true,pred)
    rmse=np.sqrt(mse)
    mae=mean_absolute_error(true,pred)
    print(f'Mean Squared Error: {mse}')
    print(f'R^2 Score: {r2}')
    print(f'Root Mean Squared Error: {rmse}')
    print(f'Mean Absolute Error: {mae}')
    return mse,r2

In [49]:
models={
    "LinearRegression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "KNeighborsRegressor": KNeighborsRegressor(),
    "DecisionTreeRegressor": DecisionTreeRegressor(),
    "AdaBoostRegressor": AdaBoostRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),
    "SVR": SVR(),
    "CatBoostRegressor": CatBoostRegressor(verbose=0),
    "XGBRegressor": XGBRegressor(eval_metric='rmse', verbosity=0)
    
}

In [50]:
for model in models:
    print(f"Training {model}...")
    models[model].fit(x_train, y_train)
    y_pred = models[model].predict(x_test)
    print(f"Results for {model}:")
    evaluate(y_test, y_pred)
    print("\n")

Training LinearRegression...
Results for LinearRegression:
Mean Squared Error: 29.095169866715516
R^2 Score: 0.8804332983749564
Root Mean Squared Error: 5.393993869732845
Mean Absolute Error: 4.214763142474852


Training Lasso...
Results for Lasso:
Mean Squared Error: 42.506416838411624
R^2 Score: 0.8253197323627852
Root Mean Squared Error: 6.519694535667421
Mean Absolute Error: 5.157881810347763


Training Ridge...
Results for Ridge:
Mean Squared Error: 29.056272192348278
R^2 Score: 0.8805931485028738
Root Mean Squared Error: 5.390387016935637
Mean Absolute Error: 4.21110068801426


Training KNeighborsRegressor...
Results for KNeighborsRegressor:
Mean Squared Error: 52.553799999999995
R^2 Score: 0.7840299763621361
Root Mean Squared Error: 7.249399975170358
Mean Absolute Error: 5.609


Training DecisionTreeRegressor...
Results for DecisionTreeRegressor:
Mean Squared Error: 60.87
R^2 Score: 0.7498545235770433
Root Mean Squared Error: 7.801922839915811
Mean Absolute Error: 6.18


Trainin